In [1]:
import os
from PIL import Image

import sys
import importlib

def call_module(nm, path):
    spec = importlib.util.spec_from_file_location(nm, path)
    foo = importlib.util.module_from_spec(spec)
    sys.modules[nm] = foo
    spec.loader.exec_module(foo)
    return foo

In [2]:
fd = "/home/jupyter/ai_font/experiments/sample"

args = call_module('args', f"{fd}/args.py")
SampleArgs = args.SampleArgs

sample = call_module('sample', f"{fd}/sample.py")
load_fontdiffuser_pipeline = sample.load_fontdiffuser_pipeline
sampling = sample.sampling

In [3]:
args = SampleArgs()
model_type = 'funit_fontdiffuser_training_phase_1'
model_i = '160000'
modelpaths = {
    'unet': f"{args.model_fd}/{model_type}__unet_{model_i}.pth",
    'content_encoder': f"{args.model_fd}/{model_type}__content_encoder_{model_i}.pth",
    'style_encoder': f"{args.model_fd}/{model_type}__style_encoder_{model_i}.pth",
}

AttributeError: 'SampleArgs' object has no attribute 'args'

In [ ]:
pipe = load_fontdiffuser_pipeline(args, args.module_fd, modelpaths)

In [17]:
args.allkorean[0].index[args.allkorean[0] == '가'].item()

0

In [7]:
args.test_fonts

['twice dahyun_4',
 'twice nayeon_6',
 'UhBee Sunhong',
 'UhBee Howl',
 'SeoulHangang Jang B',
 'MapoDPP']

In [12]:
[f for f in os.listdir(data_fd+"/seen") if ".parquet" in f]

['all_korean.parquet']

In [ ]:
for ci in content_is:
    content_image = Image.open("data/r40202/pngs/gulim__%s.png"%str(ci)).convert('RGB')
    style_image = Image.open("data/r40202/pngs/%s__%s.png"%(str(style), str(style_i))).convert('RGB')
    out_image = sampling(
        args=args, 
        pipe=pipe, 
        content_image=content_image,
        style_image=style_image,
    )